## General

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

Using TensorFlow backend.


In [0]:
from sklearn.metrics import f1_score as loss

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
df_train_raw = pd.read_csv("./Data/Train.csv")
df_test_raw = pd.read_csv("./Data/Test.csv")

In [0]:
df_train_raw.head(10)

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3
5,5,"TRACK SEASON 2016, I'M READY FOR YA @ Universi...",4
6,6,Merry Christmas you filthy little animals. Wea...,5
7,7,"Westenders @user #LAZzNation @ Weston, Toronto\n",6
8,8,Maxwell heads home @ Summa Akron City Hospital\n,2
9,9,"Story On Saturday 136 Collins Ave, Miami Beach...",7


In [0]:
df_test_raw.head(10)

,Unnamed: 0,id,TEXT
0,0,0,Thought this was cool...#Repost (get_repost)・・...
1,1,1,Happy 4th! Corte madera parade. #everytownusa ...
2,2,2,Luv. Or at least something close to it. @ Unio...
3,3,3,There's a slice of pie under that whipped crea...
4,5,5,#thankyou for your thank you We adore you both...
5,6,6,the SPECIAL4U Lyric video will be posted on my...
6,7,7,Momma Tanya's In town ! Awesome dinner @user w...
7,8,8,"Thing 1 and Thing 2 @ Huron, Ohio\n"
8,10,10,Bday girl and some random @ Sheraton New York ...
9,11,11,Always fun with my forever wedding date Congra...


In [0]:
df_test_raw.drop("Unnamed: 0", axis=1, inplace=True)
df_train_raw.drop("Unnamed: 0", axis=1, inplace=True)

In [0]:
y = df_train_raw["Label"]
df_new = df_train_raw.drop("Label", axis=1)

In [0]:
df_new.shape

(70000, 1)

In [0]:
len(df_train_raw)

70000

In [0]:
print(x_train.shape, y_train.shape)

NameError: ignored

In [0]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df_new.TEXT.values, y, 
                                                  random_state=42, 
                                                  test_size=0.3, shuffle=True)

## CountVectorize

In [0]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')
ctv.fit(list(xtrain) + list(xvalid))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
        vocabulary=None)

In [0]:
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [0]:
np.shape(xtrain_ctv)

(49000, 736444)

In [0]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

In [0]:
print ("f1loss: %0.3f " % loss(yvalid, np.argmax(predictions, axis=1), average='macro'))

## Tfidf Vectorizer

In [0]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [0]:
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [0]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("loss: %0.3f " % loss(yvalid, np.argmax(predictions, axis=1), average='macro'))

## Naive Bayes

In [0]:
# Fitting a simple Naive Bayes on TFIDF
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("loss: %0.3f " % loss(yvalid, np.argmax(predictions, axis=1), average='macro'))

In [0]:
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("loss: %0.3f " % loss(yvalid, np.argmax(predictions, axis=1), average='macro'))

## Deep Learning prep

In [0]:
!wget --header 'Host: storage.googleapis.com' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://www.kaggle.com/' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://storage.googleapis.com/kaggle-datasets/12809/17485/glove-stanford.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1549964964&Signature=daDK%2BYaMzHbexfKyvBxrpJGG9uGEgKcpgBgTuWXMw1g1b%2BvkwHK3W%2BBvBpPi34XlzI0l26K%2F3aeVhCwrnAUt8gFtLopq9XFytAr%2BKUyjcGQLTpUXgXfwmyNgDhFJyq5h3MwFGfrjcdfFipOX1C1%2B0pO0GFGNkudlM6RAgGyeU7F8Uadl1Utk%2BR8ryTlp200ukmDh0ViRBcryVTuqqS9BTsErIOxg7NOO9yFG%2FvA5ZQlQ232O1jxRt8CHptiQ3Lj9LgGdbm3m6LBZw3Pl%2F5mKkHilxd29X%2Fidrv%2FIAJZ2K53Iq7TWTuBTE1OFdSg1Q%2Fzi21M6YhGL5MRvmrxt9nph0Q%3D%3D' --output-document 'glove-stanford.zip'

--2019-02-09 09:50:02--  https://storage.googleapis.com/kaggle-datasets/12809/17485/glove-stanford.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1549964964&Signature=daDK%2BYaMzHbexfKyvBxrpJGG9uGEgKcpgBgTuWXMw1g1b%2BvkwHK3W%2BBvBpPi34XlzI0l26K%2F3aeVhCwrnAUt8gFtLopq9XFytAr%2BKUyjcGQLTpUXgXfwmyNgDhFJyq5h3MwFGfrjcdfFipOX1C1%2B0pO0GFGNkudlM6RAgGyeU7F8Uadl1Utk%2BR8ryTlp200ukmDh0ViRBcryVTuqqS9BTsErIOxg7NOO9yFG%2FvA5ZQlQ232O1jxRt8CHptiQ3Lj9LgGdbm3m6LBZw3Pl%2F5mKkHilxd29X%2Fidrv%2FIAJZ2K53Iq7TWTuBTE1OFdSg1Q%2Fzi21M6YhGL5MRvmrxt9nph0Q%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3160233772 (2.9G) [application/zip]
Saving to: ‘glove-stanford.zip’

glove-stanford.zip  100%[===================>]   2.94G  42.9MB/s    in 1m 43s  

2019-02-09 09:51:45 (29.2 MB/s) 

In [0]:
!ls

Data		     glove.840B.300d.txt	 glove.twitter.27B.200d.txt
f6e015ae-2-Data.zip  glove840b300dtxt.zip	 __MACOSX
glove.42B.300d.txt   glove-stanford.zip		 sample_data
glove.42B.300d.zip   glove.twitter.27B.100d.txt


In [0]:
embeddings_index = {}
f = open('glove.42B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1917494it [02:38, 12106.60it/s]

Found 1917494 word vectors.


In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [0]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]



  0%|          | 0/49000 [00:00<?, ?it/s]

  0%|          | 1/49000 [00:00<5:46:18,  2.36it/s]

  1%|          | 273/49000 [00:00<4:01:09,  3.37it/s]

  1%|          | 551/49000 [00:00<2:47:56,  4.81it/s]

  2%|▏         | 845/49000 [00:00<1:56:55,  6.86it/s]

  2%|▏         | 1153/49000 [00:00<1:21:24,  9.80it/s]

  3%|▎         | 1461/49000 [00:00<56:41, 13.98it/s]  

  4%|▎         | 1764/49000 [00:01<39:30, 19.93it/s]

  4%|▍         | 2076/49000 [00:01<27:32, 28.39it/s]

  5%|▍         | 2383/49000 [00:01<19:14, 40.39it/s]

  5%|▌         | 2678/49000 [00:01<13:27, 57.37it/s]

  6%|▌         | 2968/49000 [00:01<09:26, 81.26it/s]

  7%|▋         | 3257/49000 [00:01<06:38, 114.69it/s]

  7%|▋         | 3545/49000 [00:01<04:42, 161.08it/s]

  8%|▊         | 3833/49000 [00:01<03:21, 224.59it/s]

  8%|▊         | 4119/49000 [00:01<02:24, 310.36it/s]

  9%|▉         | 4413/49000 [00:01<01:45, 424.11it/s]

 10%|▉         | 4714/49000 [00:02<01:17, 571.32it/s]

 10%|█         | 5020/490

In [0]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [0]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [0]:
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [0]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [0]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector



  0%|          | 0/85530 [00:00<?, ?it/s]

 44%|████▍     | 37714/85530 [00:00<00:00, 377130.47it/s]

100%|██████████| 85530/85530 [00:00<00:00, 426515.05it/s]

## GRU

In [0]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(20))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=96, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

KeyboardInterrupt: ignored

## BiLSTM

In [0]:
xtotal, ytotal = df_new.TEXT.values, y

In [0]:
xtotal_glove = [sent2vec(x) for x in tqdm(xtotal)]
xtotal_glove_scl = scl.fit_transform(xtotal_glove)
ytotal_enc = np_utils.to_categorical(ytotal)
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(xtotal)
xtotal_seq = token.texts_to_sequences(xtotal)

# zero pad the sequences
xtotal_pad = sequence.pad_sequences(xtotal_seq, maxlen=max_len)

word_index = token.word_index

In [0]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.optimizers import Adam

model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.3, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.3)))

#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.3))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(20))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtotal_pad, y=ytotal_enc, batch_size=96, epochs=20, 
          verbose=1)

Epoch 1/20
70000/70000 [==============================] - 482s 7ms/step - loss: 2.5106
Epoch 2/20
70000/70000 [==============================] - 461s 7ms/step - loss: 2.3599
Epoch 3/20
70000/70000 [==============================] - 461s 7ms/step - loss: 2.3006
Epoch 4/20
70000/70000 [==============================] - 466s 7ms/step - loss: 2.2583
Epoch 5/20
70000/70000 [==============================] - 471s 7ms/step - loss: 2.2288
Epoch 6/20
70000/70000 [==============================] - 462s 7ms/step - loss: 2.1988
Epoch 7/20
70000/70000 [==============================] - 463s 7ms/step - loss: 2.1728
Epoch 8/20
70000/70000 [==============================] - 468s 7ms/step - loss: 2.1461
Epoch 9/20
16608/70000 [======>.......................] - ETA: 5:55 - loss: 2.1111

KeyboardInterrupt: ignored

## BiLSTM Kaggle

In [0]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.8))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(20))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 49000 samples, validate on 21000 samples
Epoch 1/100
49000/49000 [==============================] - 64s 1ms/step - loss: 2.6262 - val_loss: 2.4194
Epoch 2/100
49000/49000 [==============================] - 55s 1ms/step - loss: 2.4439 - val_loss: 2.3413
Epoch 3/100
49000/49000 [==============================] - 55s 1ms/step - loss: 2.3825 - val_loss: 2.3055
Epoch 4/100
49000/49000 [==============================] - 55s 1ms/step - loss: 2.3408 - val_loss: 2.2795
Epoch 5/100
49000/49000 [==============================] - 55s 1ms/step - loss: 2.3016 - val_loss: 2.2653
Epoch 6/100
49000/49000 [==============================] - 55s 1ms/step - loss: 2.2773 - val_loss: 2.2477
Epoch 7/100
49000/49000 [==============================] - 54s 1ms/step - loss: 2.2506 - val_loss: 2.2364
Epoch 8/100
49000/49000 [==============================] - 54s 1ms/step - loss: 2.2236 - val_loss: 2.2364
Epoch 9/100
49000/49000 [==============================] - 55s 1ms/step - loss: 2.2075 - val_loss: 2.2

## Predictions

In [0]:
preds = model.predict(xvalid_pad)

In [0]:
preds.shape

(21000, 20)

Test 1 = 0.18446249324196778 ( dropout - 0.8, no 200 layer) \\
Test 2 = 0.214 \\
Test 3 = 0.22239185191649283 \\
Test4 kaggle = 0.212

In [0]:
loss(yvalid, np.argmax(preds, axis=1), average='macro')

0.21260058311881092

In [0]:
df = pd.read_csv("./Data/Test.csv")

In [0]:
xtest = df.TEXT.values

In [0]:
xtest_glove = [sent2vec(x) for x in tqdm(xtest)]
xtest_glove = np.array(xtest_glove)
xtest_glove_scl = scl.transform(xtest_glove)
xtest_seq = token.texts_to_sequences(xtest)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)



  0%|          | 0/25958 [00:00<?, ?it/s]

  1%|          | 297/25958 [00:00<00:08, 2964.91it/s]

  2%|▏         | 593/25958 [00:00<00:08, 2962.24it/s]

  3%|▎         | 900/25958 [00:00<00:08, 2993.59it/s]

  5%|▍         | 1207/25958 [00:00<00:08, 3013.31it/s]

  6%|▌         | 1493/25958 [00:00<00:08, 2965.41it/s]

  7%|▋         | 1790/25958 [00:00<00:08, 2964.37it/s]

  8%|▊         | 2061/25958 [00:00<00:08, 2880.43it/s]

  9%|▉         | 2348/25958 [00:00<00:08, 2876.60it/s]

 10%|█         | 2635/25958 [00:00<00:08, 2873.81it/s]

 11%|█▏        | 2926/25958 [00:01<00:07, 2883.66it/s]

 12%|█▏        | 3231/25958 [00:01<00:07, 2930.53it/s]

 14%|█▎        | 3533/25958 [00:01<00:07, 2955.96it/s]

 15%|█▍        | 3830/25958 [00:01<00:07, 2958.52it/s]

 16%|█▌        | 4127/25958 [00:01<00:07, 2958.18it/s]

 17%|█▋        | 4431/25958 [00:01<00:07, 2982.21it/s]

 18%|█▊        | 4732/25958 [00:01<00:07, 2987.67it/s]

 19%|█▉        | 5030/25958 [00:01<00:07, 2898.18it/s]

 21%|█

In [0]:
preds = model.predict(xtest_pad)

In [0]:
df.drop("Unnamed: 0", axis=1, inplace=True)
df.drop("TEXT", axis=1, inplace=True)

In [0]:
df["Label"] = np.argmax(preds, axis=1)

In [0]:
df.to_csv("Sub8fullhari.csv", index=False)